In [28]:
%load_ext autoreload
%autoreload 2
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from utilities import data_handler
import numpy as np
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer # for tokenization only

# THE CELL BELLOW CONTAINS THE FILENAMES TO CHANGE

In [84]:
UNLABLED_DATA = '/Users/ianmagnusson/IITUDND/data/retrieved_data/tweets/mexico_earthquake_extras.json'
LABLED_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/json/mexico_earthquake_final_data.json'
CLASS_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/annotations/mexico_earthquake_final_data.tsv'
NPY_OUTPUT_FILE = '/Users/ianmagnusson/IITUDND/data/extracted_features/glove/preprocessed/mexico.npz'

## Set up glove

In [9]:
EMBED_DIM = 200
conversion_file = '../models/gensim_glove.txt'
# convert glove format to work with gensim. tutorial here https://radimrehurek.com/gensim/scripts/glove2word2vec.html
# _ = glove2word2vec('/Users/ianmagnusson/IITUDND/models/glove.twitter.27B.200d.txt', conversion_file)

In [4]:
# load model, NOTE this is very slow!
glove = KeyedVectors.load_word2vec_format(conversion_file)

# Extract glove embeddings

In [85]:
data = data_handler.DataHandler(UNLABLED_DATA,LABLED_DATA,CLASS_DATA)


In [49]:
def create_id_2_embed(tweet_jsons):
    id_2_embed = {}
    tokenizer = CountVectorizer().build_tokenizer()
    for i, tweet_json in enumerate(tweet_jsons):
        embedded = np.zeros(EMBED_DIM)
        text = tweet_json['text'].lower()
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                embedded += glove[token]
                num_in_vocab += 1
        if str(tweet_json['id']) in id_2_embed:
            print('oops')
        if num_in_vocab:
            embedded = embedded / num_in_vocab
        id_2_embed[str(tweet_json['id'])] = embedded
        
    return id_2_embed

In [86]:
id_2_embed = create_id_2_embed(data.merged)

In [87]:
len(id_2_embed)

3521

In [88]:
# save as a dictionary {filename : np.array}

with open(NPY_OUTPUT_FILE, 'wb+') as fout:
    np.savez_compressed(fout, **id_2_embed)